In [12]:
# import os
import glob
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
from pprint import pprint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

from smartcar.utils.path import get_data_paths
from smartcar.utils.read import read_json_label

In [8]:
data_dir = "C:\Projects\SmartCar\data"
image_fnames, label_fnames = get_data_paths(data_dir)

batch_size = 16

In [216]:
class SmartCarGenerator(Sequence):
    def __init__(self, image_fnames, label_fnames, batch_size, image_shape, shuffle=True):
        self.image_fnames = np.array(image_fnames)
        self.label_fnames = np.array(label_fnames)
        self.batch_size = batch_size
        self.image_shape = image_shape
    
        if shuffle:
            indices = np.array([i for i in range(len(image_fnames))])
            indices = np.random.permutation(indices)
            self.image_fnames = self.image_fnames[indices]
            self.label_fnames = self.label_fnames[indices]
        
    def __len__(self):
        return np.ceil(len(image_fnames) / float(self.batch_size)).astype(np.int)
                       
    def __getitem__(self, idx):
        x_fnames = self.image_fnames[idx * self.batch_size : (idx+1) * self.batch_size]
        y_fnames = self.label_fnames[idx * self.batch_size : (idx+1) * self.batch_size]
        
        batch_x = np.array([cv2.resize(cv2.imread(path), (self.image_shape[1], self.image_shape[0]), interpolation=cv2.INTER_NEAREST) for path in x_fnames])
        return batch_x

In [217]:
datagen = SmartCarGenerator(image_fnames, label_fnames, batch_size=16, image_shape=(120, 160, 3), shuffle=True)

In [218]:
batch_x = datagen.__getitem__(0)

In [ ]:
from random import shuffle

def augment_brightness(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = .5 + np.random.uniform()
    image1[:,:,2] = image1[:,:,2] * random_bright
    image1[:,:,2][image1[:,:,2] > 255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def build_dataset(datas):
    dataset = np.zeros((len(datas), 120, 160, 3))
    for i in range(len(datas)):
        img = cv2.imread(datas[i])
        img = augment_brightness(img)
        y1 = int(img.shape[0] / 2)
        img = img[y1:, :]
        img = cv2.resize(img, (160, 120), interpolation = cv2.INTER_CUBIC)
        dataset[i, :, :, :] = img
    return dataset

def myGenerator(datas, angle_labels, speed_labels, batch=16):
    # For each epoch
    while 1:
        # Suffle datas
        indexes_tmp = list(range(len(datas)))
        indexes = []
        for j in range(3):
            indexes += indexes_tmp
        shuffle(indexes)
        
        # For each batch
        for i in range(0, len(indexes), batch):
            
            # Build a batch of datas
            batch_data = [datas[indexes[j]] for j in range(i, i + batch) if j < len(indexes)]
            y_angle = [angle_labels[indexes[j]] for j in range(i, i + batch) if j < len(indexes)]
            y_speed = [speed_labels[indexes[j]] for j in range(i, i + batch) if j < len(indexes)]
    
            y_angle = np.array(y_angle)
            y_speed = np.array(y_speed)
            # Build the dataset for the batch
            x = build_dataset(batch_data)
            
            yield x, [y_angle, y_speed]

In [ ]:
sep_train = int(nb_data * 0.8)
sep_test = nb_data - sep_train
gen_train = myGenerator(image_paths[:sep_train], angle_labels[:sep_train], speed_labels[:sep_train], BATCH_SIZE)
gen_test = myGenerator(image_paths[sep_train:], angle_labels[sep_train:], speed_labels[sep_train:], BATCH_SIZE)

In [ ]:
plt.hist(angle_labels, bins=50)
plt.show()

In [ ]:
plt.hist(speed_labels, bins=30)
plt.show()

In [ ]:
def AnglePredCnn():
    img_in = Input(shape=(120, 160, 3), name='img_in')                      # First layer, input layer, Shape comes from camera.py resolution, RGB
    x = img_in
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)       # 24 features, 5 pixel x 5 pixel kernel (convolution, feauture) window, 2wx2h stride, relu activation
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)       # 32 features, 5px5p kernel window, 2wx2h stride, relu activatiion
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)       # 64 features, 5px5p kernal window, 2wx2h stride, relu
    x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)       # 64 features, 3px3p kernal window, 2wx2h stride, relu
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)       # 64 features, 3px3p kernal window, 1wx1h stride, relu

    # Possibly add MaxPooling (will make it less sensitive to position in image).  Camera angle fixed, so may not to be needed

    x = Flatten(name='flattened')(x)                                        # Flatten to 1D (Fully connected)
    x = Dense(100, activation='linear')(x)                                    # Classify the data into 100 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out (turn off) 10% of the neurons (Prevent overfitting)
    x = Dense(50, activation='linear')(x)                                     # Classify the data into 50 features, make all negatives 0
    x = Dropout(.1)(x)                                                      # Randomly drop out 10% of the neurons (Prevent overfitting)
    #categorical output of the angle
    angle_out = Dense(1, activation='linear', name='angle_out')(x)        # Connect every input with every output and output 15 hidden units. Use Softmax to give percentage. 15 categories and find best one based off percentage 0.0-1.0
    
    #continous output of throttle
    throttle_out = Dense(1, activation='linear', name='throttle_out')(x)      # Reduce to 1 number, Positive number only
    #out = Dense(2, activation='linear', name='model_outputs')(x)
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])#outputs=[out])
    model.compile(optimizer='adam',
                  loss={'angle_out': 'mean_squared_error',
                        'throttle_out': 'mean_squared_error'},
                  #loss={'model_outputs' : 'mean_squared_error'})
                  loss_weights={'angle_out': 0.9, 'throttle_out': .001})
    return model

In [ ]:
model = AnglePredCnn()

In [ ]:
model_path = os.path.abspath("models/model_aug_bright.h5")
checkpointer = ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss', mode='min')
hist = model.fit_generator(generator=gen_train,
                           validation_data=gen_test,
                           steps_per_epoch=(sep_train * 3 / BATCH_SIZE),
                           validation_steps=(sep_test * 3 / BATCH_SIZE),
                           epochs=10,
                           shuffle=True,
                           callbacks=[checkpointer])

In [ ]:
plt.plot(hist.history["loss"][1:])
plt.plot(hist.history["val_loss"][1:])
plt.show()